In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

For the meaning of sunspot   [Click](https://en.wikipedia.org/wiki/Sunspot)

![Sunspot](http://i2.wp.com/thegrandsolarminimum.com/wp-content/uploads/2019/03/Sun_sunspot.jpg?ssl=1)

In [ ]:
sspot=pd.read_csv('/kaggle/input/daily-sun-spot-data-1818-to-2019/sunspot_data.csv')

# Data Analysis(EDA)

In [ ]:
sspot

Drop the unnecessary column 

In [ ]:
sspot.drop("Unnamed: 0", axis=1, inplace=True)

In [ ]:
sspot.sample(5)

In [ ]:
sspot.dtypes

If Number of Sunspots value is -1, it means the missing value.

In [ ]:
sspot[(sspot["Number of Sunspots"]== -1 )]#3247 missing values in 73718 rows

In [ ]:
df=sspot.copy()

Missing values in number of sunspots column are in minority so I decided to drop them.(3247 to 73718)

In [ ]:
nan_value = float("NaN")

df.replace(-1,nan_value, inplace=True)

df

In [ ]:
df = df.dropna()
df = df.reset_index(drop=True)

After the cleaning:

In [ ]:
df

Checking the Null Values

In [ ]:
df.isnull().sum()

In [ ]:
df.groupby("Year").agg(["min", "max", "std", "mean"])

**Which year was the most seen sunspot?**

In [ ]:
a=df.groupby(['Year']).sum()
t=a.sort_values(by='Number of Sunspots', ascending = False)
t['Number of Sunspots']

> **Answer is 1957**

# Data Visualization

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

Increase in the Number of Obsevations

In [ ]:
cmap = sns.cubehelix_palette(dark=.3, light=.9, as_cmap=True)
plt.figure(figsize=(20,20))
sns.scatterplot(data=df, x="Number of Sunspots", y="Year", hue="Observations", palette=cmap)
plt.show()

It's interesting ,because while the nums of observation increasing, observation of sunpots decreased.

In [ ]:
t['Number of Sunspots'].head(5).plot(kind='barh', figsize=(10,10))
yaxes="Number of Sunpots"

# Feature Engineering

In [ ]:
dfcorr=df.corr()
dfcorr

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(dfcorr, cmap='coolwarm')

In [ ]:
df

I analyzed the correlation matrix and after that I choosed most necessary columns for prediction to Sunspots num.

In [ ]:
x = df.drop(["Number of Sunspots","Month","Day"],axis=1)

In [ ]:
x

In [ ]:
y= df["Number of Sunspots"]
y=y.to_frame()
y

# Model Setup

# Split Data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25, random_state=0)
x_train.shape

In [ ]:
y_train.shape

# Model Choosing

### Neural Network 

In [ ]:
from sklearn.neural_network import MLPRegressor
neu = MLPRegressor(random_state=1, max_iter=450).fit(x_train, y_train)

In [ ]:
df_new=pd.DataFrame(neu.predict(x_test))
pred_val= df_new.rename(columns={0: 'Number of Sunspots'})
pred_val

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
print("MLPRegressor's R2 Score:",r2_score(y_test, pred_val))

### Decision Trees

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeRegressor()
clf = clf.fit(x_train, y_train)

In [ ]:
df_new=pd.DataFrame(clf.predict(x_test))
pred_val= df_new.rename(columns={0: 'Number of Sunspots'})
pred_val

In [ ]:
print("Decision Tree Regressor's R2 Score:",r2_score(y_test, pred_val))

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

**After trials I choosed Decision Tree.**

# Performance Metrics

In [ ]:
mean_absolute_error(y_test, pred_val)

In [ ]:
pred_val.head(5)

In [ ]:
y_test.head(5)

# To_CSV

In [ ]:
pred_val.to_csv('sunspotprediction.csv', index=False)